**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4034 - Análisis de grandes volúmenes de datos**

Tecnológico de Monterrey

**Semana 4**

Avance de proyecto 1: Sistema de Recomendación

**Equipo 20**

  Zmaltzin Paola Godínez Juárez - A01794538

  Julio César Mestas Delgado    - A01794827

---

In [ ]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from node2vec import Node2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importando los datos
df_netflix = pd.read_csv("/content/drive/MyDrive/Documentos/Tec de Monterrey/MNA/BIG DATA/SEMANA 5/version 2/version 2/archive/netflix_titles.csv")
df_netflix = df_netflix.dropna()    # Eliminando valores nulos
df_netflix.drop(                    # Eliminando columnas que no se utilizarán
    columns=[
        "director",
        "cast",
        "country",
        "date_added",
        "release_year",
        "rating",
        "duration",
        "type",
    ],
    inplace=True,
)
# Imprimiendo los primeros 5 renglones del dataframe
df_netflix.head(5)

<ipython-input-4-19a3531dfc3f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_netflix.drop(                    # Eliminando columnas que no se utilizarán


,show_id,title,listed_in,description
7,s8,Sankofa,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,The Great British Baking Show,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,The Starling,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Je Suis Karl,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Jeans,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...


In [ ]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5332 entries, 7 to 8806
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   show_id      5332 non-null   object
 1   title        5332 non-null   object
 2   listed_in    5332 non-null   object
 3   description  5332 non-null   object
dtypes: object(4)
memory usage: 208.3+ KB


In [ ]:
# Evaluando valores faltantes
faltantes = df_netflix.isnull().sum()*100/len(df_netflix)
faltantes_df = pd.DataFrame({'% de valores faltantes': faltantes})
faltantes_df

,% de valores faltantes
show_id,0.0
title,0.0
listed_in,0.0
description,0.0


In [ ]:
# Función para crear una gráfica con los títulos de películas y sus generos como nodos
def addToGraph(movie_name, grafica):
    generos = (
        df_netflix[df_netflix["title"] == movie_name]["listed_in"]
        .values[0]
        .rstrip()
        .lower()
        .split(", ")
    )
    for genero in generos:
        grafica.add_edge(movie_name.strip(), genero)
    return grafica


# Función para crear la gráfica para todos los nombres de películas
def crearGrafica():
    grafica = nx.Graph()
    for movie_name in df_netflix["title"]:
        grafica = addToGraph(movie_name, grafica)
    return grafica

In [ ]:
grafica = crearGrafica()

In [ ]:
# Imprimir los generos asociados a una película
print('Generos asociados a la pelicula Norm of the North: King Sized Adventure:', grafica.degree()["Norm of the North: King Sized Adventure"])
print('Generos asociados a la pelicula #realityhigh:', grafica.degree()["#realityhigh"])

Generos asociados a la pelicula Norm of the North: King Sized Adventure: 2
Generos asociados a la pelicula #realityhigh: 1


# Algoritmo Node2vec

Node2vec es un algoritmo de incrustación de nodos que calcula una representación vectorial de un nodo basándose en recorridos de gráficos aleatorios. Funciona de la misma manera que Word2Vec, excepto que en lugar de incrustaciones de palabras, se crean incrustaciones de nodos.


In [ ]:
# Estrategia de Node2vec y parametros utilizados
# Dimensiones del embedding = 20
# Número de walks aleatorios que se generarán para cada nodo del gráfico: 10
node2vec = Node2Vec(grafica, dimensions=20, walk_length=16, num_walks=10)

Computing transition probabilities:   0%|          | 0/5373 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [01:09<00:00,  6.93s/it]


In [ ]:
# Entrenando el modelo
model = node2vec.fit(window=5, min_count=1)

In [ ]:
# Revisar el embedding de la película 'El conjuro'
model.wv.get_vector("The Conjuring")

array([-0.11661051,  0.27653116, -0.48582515,  0.00502838,  0.6468146 ,
        0.07118495,  0.53167546,  0.20060545, -0.29165465,  1.1979458 ,
        0.10287121, -0.54496074,  0.5583805 ,  0.045446  ,  0.7088728 ,
       -0.4914866 ,  0.7928158 ,  0.8006262 , -0.64038545, -1.2228585 ],
      dtype=float32)

In [ ]:
# Revisar el embedding de la película 'Barbie Big City Big Dreams'
model.wv.get_vector("Jaws")

array([ 0.67564017, -0.3557283 ,  0.01856837, -0.3658011 ,  0.24912146,
        0.02840949,  0.44205186,  0.12706602,  0.4922119 ,  0.01891412,
        0.22500034,  0.20394203, -0.5427183 , -0.64250594,  0.34111658,
        0.3280051 ,  1.5767963 , -0.04673437, -0.227651  , -1.0185492 ],
      dtype=float32)

In [ ]:
# Función para generar películas similares para un genero o título
def node2vec_recommender(name):
    for node, _ in model.wv.most_similar(name):
        print(node)

In [ ]:
# Recomendaciones para la película 'El conjuro'
node2vec_recommender("The Conjuring")

What Lies Beneath
The Strangers
The Bye Bye Man
Unfriended
Our House
Delirium
Desolation
House at the End of the Street
Hush
The Open House


In [ ]:
# Recomendaciones para la película 'Ex Machina'
node2vec_recommender("Ex Machina")

A Sun
Budhia Singh: Born to Run
The Disciple
Chauthi Koot
Jaoon Kahan Bata Ae Dil
Kalel, 15
Zoo
Kia and Cosmos
Once Again
The Little Wars


# Comentarios

Las recomendaciones obtenidas mediante este método muestran consistencia considerando el género. Por ejemplo para la película de El Conjuro como Delirium o Desolation. Para el caso de Ex-Maquina, Kia and Cosmos podría considerarse una buena recomendación.